# Setup

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
os.environ["CUDA_LAUNCH_BLOCKING"]="1, 2, 3, 4"

In [2]:
import numpy as np
import pandas as pd

from sklearn.metrics.pairwise import cosine_similarity

In [3]:
import torch
from sklearn.preprocessing import normalize
from torch.utils.data import Dataset, DataLoader
from torchsampler import ImbalancedDatasetSampler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import f1_score, precision_score, recall_score

# Import Data

In [4]:
items = pd.read_csv("../Datasets/ml-100k/Text/items.csv")

r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv('../Datasets/ml-100k/Text/u1.base', sep='\t', names=r_cols,encoding='latin-1')

In [5]:
# n_users = ratings.user_id.unique().shape[0]
# n_items = ratings.movie_id.unique().shape[0]
n_users = 943
n_items = 1682

In [6]:
data_matrix = np.zeros((n_users, n_items))
for line in ratings.itertuples():
    data_matrix[line[1]-1, line[2]-1] = line[3]

In [7]:
data_matrix_emp = data_matrix.copy()
data_matrix_emp[data_matrix < 4] = 0
data_matrix_emp[data_matrix >= 4]= 1 

In [8]:
train_indices = list(zip(*(np.where(data_matrix != 0))))

In [9]:
len(train_indices)

80000

# Siamese network

## Data loader

In [10]:
torch.cuda.is_available()

True

In [11]:
torch.__version__

'1.10.1'

In [12]:
items_csv = "../Datasets/ml-100k/Text/items.csv"
train_ratings = "../Datasets/ml-100k/Text/u1.base"
test_ratings = "../Datasets/ml-100k/Text/u1.test"

item_path = "../Datasets/ml-100k/"

In [13]:
# cosine_similarity(np.nan_to_num(pd.read_csv(item_path)))

In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [15]:
class MovielensDataset(Dataset):
    def __init__(self, ratings = train_ratings, item_path = item_path, device = device):
        self.video_embeddings = pd.read_csv(item_path + "Video/embeddings.csv").to_numpy()
        self.audio_embeddings = pd.read_csv(item_path + "Audio/embeddings.csv").to_numpy()
        self.text_embeddings = pd.read_csv(item_path + "Meta/embeddings.csv").to_numpy()
        self.ratings = pd.read_csv(ratings, sep='\t', 
                                   names=['user_id', 'movie_id', 'rating', 'unix_timestamp'],encoding='latin-1')
        self.indices = None
        self.device = device
        self.data = None
        self.n_users = None
        self.n_items = None
        self.fill_ratings()
        self.embeddings()
    
    def fill_ratings(self, threshold=4):
        self.n_users = self.ratings.user_id.unique().shape[0]
        self.n_items = self.ratings.movie_id.unique().shape[0]
        
        self.data = np.zeros((n_users, n_items))
        for line in self.ratings.itertuples():
            self.data[line[1]-1, line[2]-1] = line[3]
            
        self.indices = list(zip(*(np.where(self.data != 0))))
            
    def embeddings(self):
        self.audio_embeddings = np.nan_to_num(self.audio_embeddings)
        self.video_embeddings = np.nan_to_num(self.video_embeddings)
        self.audio_embeddings = normalize(self.audio_embeddings, axis = 1)
#         self.item_embeddings = np.append(data_matrix.T, self.item_embeddings, axis = 1)
        self.user_embeddings = data_matrix
        self.item_embeddings = data_matrix.T
        self.video_embedding_size = self.video_embeddings.shape[1]
        self.audio_embedding_size = self.audio_embeddings.shape[1]
        self.text_embedding_size = self.text_embeddings.shape[1]
        self.user_embedding_size = self.user_embeddings.shape[1]
        self.item_embedding_size = self.item_embeddings.shape[1]
        
    def __len__(self):
        return(len(self.indices))
    
    def __getitem__(self, idx):
        user = self.indices[idx][0]
        item = self.indices[idx][1]

        xu = torch.from_numpy(self.user_embeddings[user]).to(self.device)
        xv = torch.from_numpy(self.video_embeddings[item]).to(self.device)
        xa = torch.from_numpy(self.audio_embeddings[item]).to(self.device)
        xt = torch.from_numpy(self.text_embeddings[item]).to(self.device)
        xi = torch.from_numpy(self.item_embeddings[item]).to(self.device)
        
        y = self.data[user][item]
        return(xu.float(), [xv.float(), xa.float(), xt.float(), xi.float()], int(y))

In [16]:
train_dataset = MovielensDataset(ratings = train_ratings)
test_dataset = MovielensDataset(ratings = test_ratings)

/home/sriram_1801cs37/anaconda3/envs/pytorch/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/sriram_1801cs37/anaconda3/envs/pytorch/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/sriram_1801cs37/anaconda3/envs/pytorch/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/home/sriram_1801cs37/anaconda3/envs/pytorch/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [17]:
target = [int(data_matrix[train_dataset.indices[i][0]][train_dataset.indices[i][1]]) - 1 for i in range(len(train_dataset))]

In [18]:
class_sample_count = np.unique(data_matrix, return_counts=True)[1][1::]

weights = 1 / torch.Tensor(class_sample_count)
weights = weights.double()
weights_all = weights[target]

In [19]:
sampler = torch.utils.data.sampler.WeightedRandomSampler(weights_all, len(weights_all))

In [20]:
trainloader = DataLoader(train_dataset, batch_size = 30000, shuffle = True)
testloader = DataLoader(test_dataset, batch_size = 20000, shuffle = False)

## Architecture

In [21]:
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import OrderedDict

In [22]:
from ml_metrics import mapk
from recmetrics import mark

In [23]:
class SiameseNet(nn.Module):
    def __init__(self, device = device):
        super(SiameseNet, self).__init__()
        self.encoder_user = nn.Sequential(OrderedDict([
            ('linr1', nn.Linear(train_dataset.user_embedding_size, 1024)),
            ('relu1', nn.LeakyReLU()),
            ('linr2', nn.Linear(1024, 200)),
            ('relu2', nn.LeakyReLU()),
#             ('linr3', nn.Linear(800, 500)),
#             ('relu3', nn.LeakyReLU()),
#             ('linr4', nn.Linear(500, 600)),
#             ('relu4', nn.LeakyReLU()),
        ]))
        
        self.encoder_item = nn.Sequential(OrderedDict([
            ('linr1', nn.Linear(train_dataset.item_embedding_size, 256)),
            ('relu1', nn.Sigmoid()),
            ('linr2', nn.Linear(256, 200)),
            ('relu2', nn.LeakyReLU()),
        ]))
        
        self.encoder_video = nn.Sequential(OrderedDict([
            ('linr1', nn.Linear(train_dataset.video_embedding_size, 1600)),
            ('relu1', nn.Sigmoid()),
            ('linr2', nn.Linear(1600, 1000)),
            ('relu2', nn.Sigmoid()),
            ('linr3', nn.Linear(1000, 500)),
            ('relu3', nn.LeakyReLU()),
            ('linr4', nn.Linear(500, 200)),
            ('relu4', nn.LeakyReLU()),
        ]))
        
        self.encoder_audio = nn.Sequential(OrderedDict([
            ('linr1', nn.Linear(train_dataset.audio_embedding_size, 256)),
            ('relu1', nn.Sigmoid()),
            ('linr2', nn.Linear(256, 200)),
            ('relu2', nn.LeakyReLU()),
        ]))
        
        self.encoder_text = nn.Sequential(OrderedDict([
            ('linr1', nn.Linear(train_dataset.text_embedding_size, 100)),
            ('relu1', nn.LeakyReLU()),
            ('linr2', nn.Linear(100, 200)),
            ('relu2', nn.LeakyReLU()),
        ]))
        
        self.siamese = nn.Sequential(OrderedDict([
            ('linr1', nn.Linear(200, 512)),
            ('relu1', nn.LeakyReLU()),
            ('linr2', nn.Linear(512, 256)),
            ('relu2', nn.LeakyReLU()),
            ('linr3', nn.Linear(256, 100)),
            ('relu3', nn.LeakyReLU()),
        ]))
        
        self.ffn = nn.Sequential(OrderedDict([
            ('linr1', nn.Linear(200, 164)),
            ('actv1', nn.LeakyReLU()),
            ('linr2', nn.Linear(164, 50)),
            ('actv2', nn.LeakyReLU()),
            ('linr4', nn.Linear(50, 1)),
        ]))
        
        self.device = device
        self.encoder_user.apply(self.init_weights)
        self.encoder_item.apply(self.init_weights)
        self.encoder_video.apply(self.init_weights)
        self.encoder_audio.apply(self.init_weights)
        self.encoder_text.apply(self.init_weights)
        self.siamese.apply(self.init_weights)
        self.ffn.apply(self.init_weights)
        
    def init_weights(self, m):
        if isinstance(m, nn.Linear):
            nn.init.kaiming_normal_(m.weight)
            m.bias.data.fill_(0.01)
        
    def forward(self, x1, x2):
        outu = self.encoder_user(x1)
        outr = self.encoder_item(x2[3])
        outv = self.encoder_video(x2[0])
        outa = self.encoder_audio(x2[1])
        outt = self.encoder_text(x2[2])
#         outi = torch.cat((outt), axis=1)
        outi = outt
        out1 = self.siamese(outu)
        out2 = self.siamese(outi)
        diff = torch.cat((out1, out2), axis=1)
        out = self.ffn(diff)
        return(out, outu, outi)
    
    def fit(self, trainloader = trainloader, 
            testloader = testloader, epochs = 200):
        self.criterion_rate = nn.MSELoss()
        self.criterion_embd = nn.CosineEmbeddingLoss()
        self.optimizer = optim.Adam(self.parameters(), lr = 5e-4, weight_decay = 1e-4)
        
        train_loss = []
        train_f1 = []
        test_loss = []
        test_f1 = []
        for epoch in range(epochs):
            running_loss = 0.0
            running_loss_1 = 0.0
            
            for i, data in tqdm(enumerate(trainloader)):
                self.train()
                x1, x2, y = data
                y_flt = y.type(torch.FloatTensor).to(device)
                y_lng = torch.div(y, 4, rounding_mode="floor").to(device)
                self.optimizer.zero_grad()
                reg, outu, outi = self.forward(x1, x2)
                loss_1 = self.criterion_rate(reg.squeeze(), y_flt)
                loss_2 = self.criterion_embd(outu, outi, y_lng * 2 - 1)
                loss = loss_1 + loss_2 
                loss.backward()
                self.optimizer.step()

                running_loss_1 += torch.sqrt(loss_1)
                running_loss += loss
            vl, vp, vr, vf, tp, tr, tf = self.evaluate()
            print('Epoch-%d: Loss = %.3f\nTrain RMSE = %.3f||Train Precision = %.3f||Train Recall = %.3f\nTest RMSE = %.3f || Test Precision = %.3f|| Test Recall = %.3f'%
                  (epoch + 1, running_loss / i, running_loss_1 / i, 
                   tp, tr, vl, vp, vr))
            train_loss.append((running_loss_1 / i).cpu().detach().numpy())
            test_loss.append(vl.cpu().detach().numpy())
            train_f1.append(tf)
            test_f1.append(vf)
        return(train_loss, test_loss, train_f1, test_f1)
            
    def evaluate(self, k = 3.35):
        self.eval()
        with torch.no_grad():
            valdata = next(iter(testloader))
            x1, x2, y = valdata
            y_flt = y.type(torch.FloatTensor).to(device)
            y_lng = torch.div(y, 4, rounding_mode="floor").to(device)
            otpt = self.forward(x1, x2)
            print(otpt[0])
            pred = (otpt[0] > k).float()
            vl = torch.sqrt(self.criterion_rate(otpt[0].squeeze(), y_flt))
            vp = precision_score(y_lng.cpu(), pred.cpu())
            vr = recall_score(y_lng.cpu(), pred.cpu())
            vf = f1_score(y_lng.cpu(), pred.cpu())
            print(classification_report(y_lng.cpu(), pred.cpu(),
                    target_names = ["0", "1"]))
            
            traindata = next(iter(trainloader))
            x1, x2, y = traindata
            y_flt = y.type(torch.FloatTensor).to(device)
            y_lng = torch.div(y, 4, rounding_mode="floor").to(device)
            otpt = self.forward(x1, x2)
            pred = (otpt[0] > k).float()
            tp = precision_score(y_lng.cpu(), pred.cpu())
            tr = recall_score(y_lng.cpu(), pred.cpu())
            tf = f1_score(y_lng.cpu(), pred.cpu())
        return(vl, vp*100, vr*100, vf*100, tp*100, tr*100, tf*100)

In [24]:
sm_net = SiameseNet()
sm_net.to(device)

SiameseNet(
  (encoder_user): Sequential(
    (linr1): Linear(in_features=1682, out_features=1024, bias=True)
    (relu1): LeakyReLU(negative_slope=0.01)
    (linr2): Linear(in_features=1024, out_features=200, bias=True)
    (relu2): LeakyReLU(negative_slope=0.01)
  )
  (encoder_item): Sequential(
    (linr1): Linear(in_features=943, out_features=256, bias=True)
    (relu1): Sigmoid()
    (linr2): Linear(in_features=256, out_features=200, bias=True)
    (relu2): LeakyReLU(negative_slope=0.01)
  )
  (encoder_video): Sequential(
    (linr1): Linear(in_features=2048, out_features=1600, bias=True)
    (relu1): Sigmoid()
    (linr2): Linear(in_features=1600, out_features=1000, bias=True)
    (relu2): Sigmoid()
    (linr3): Linear(in_features=1000, out_features=500, bias=True)
    (relu3): LeakyReLU(negative_slope=0.01)
    (linr4): Linear(in_features=500, out_features=200, bias=True)
    (relu4): LeakyReLU(negative_slope=0.01)
  )
  (encoder_audio): Sequential(
    (linr1): Linear(in_featur

In [ ]:
train_loss, test_loss, train_f1, test_f1 = sm_net.fit()

3it [00:25,  8.64s/it]


tensor([[1.6902],
        [1.8574],
        [1.6902],
        ...,
        [2.1537],
        [2.0538],
        [2.0912]], device='cuda:0')
              precision    recall  f1-score   support

           0       0.44      0.94      0.60      8765
           1       0.55      0.06      0.10     11235

    accuracy                           0.44     20000
   macro avg       0.49      0.50      0.35     20000
weighted avg       0.50      0.44      0.32     20000

Epoch-1: Loss = 10.919
Train RMSE = 3.746||Train Precision = 57.072||Train Recall = 14.610
Test RMSE = 1.854 || Test Precision = 55.095|| Test Recall = 5.679


3it [00:21,  7.24s/it]


tensor([[3.1411],
        [3.3875],
        [3.1411],
        ...,
        [3.3955],
        [3.3239],
        [3.2850]], device='cuda:0')
              precision    recall  f1-score   support

           0       0.45      0.50      0.48      8765
           1       0.58      0.53      0.55     11235

    accuracy                           0.52     20000
   macro avg       0.52      0.52      0.51     20000
weighted avg       0.52      0.52      0.52     20000

Epoch-2: Loss = 4.308
Train RMSE = 2.294||Train Precision = 56.706||Train Recall = 67.604
Test RMSE = 1.449 || Test Precision = 57.666|| Test Recall = 53.164


3it [00:21,  7.23s/it]


...

3it [00:22,  7.34s/it]


tensor([[4.0109],
        [4.1250],
        [4.0109],
        ...,
        [3.4984],
        [3.7142],
        [3.5564]], device='cuda:0')
              precision    recall  f1-score   support

           0       0.71      0.58      0.64      8765
           1       0.71      0.82      0.76     11235

    accuracy                           0.71     20000
   macro avg       0.71      0.70      0.70     20000
weighted avg       0.71      0.71      0.71     20000

Epoch-129: Loss = 1.754
Train RMSE = 1.336||Train Precision = 72.070||Train Recall = 82.516
Test RMSE = 0.961 || Test Precision = 71.488|| Test Recall = 81.656


3it [00:21,  7.13s/it]


tensor([[4.1556],
        [4.3682],
        [4.1556],
        ...,
        [3.5467],
        [3.8709],
        [3.7589]], device='cuda:0')
              precision    recall  f1-score   support

           0       0.76      0.49      0.60      8765
           1       0.69      0.88      0.77     11235

    accuracy                           0.71     20000
   macro avg       0.72      0.68      0.68     20000
weighted avg       0.72      0.71      0.70     20000

Epoch-130: Loss = 1.760
Train RMSE = 1.339||Train Precision = 69.011||Train Recall = 88.762
Test RMSE = 0.963 || Test Precision = 68.876|| Test Recall = 87.850


0it [00:00, ?it/s]

In [ ]:
torch.save(sm_net, "./pretrained/meta.pth")

In [ ]:
import matplotlib.pyplot as plt

plt.plot(train_loss[10::], label = "Train Loss")
plt.plot(test_loss[10::], label = "Test Loss")
plt.legend()
plt.show()
plt.plot(train_f1, label = "Train F1")
plt.plot(test_f1, label = "Test F1")
plt.legend()
plt.show()


# Test

In [ ]:
import matplotlib.pyplot as plt
import torch

In [ ]:
sm_net = torch.load("./pretrained/meta.pth")

In [ ]:
prs = []
rec = []
f1 = []

for i in np.arange(3, 4, 0.1):
    _, p, r, f, _, _, _ = sm_net.evaluate(i)
    prs.append(p)
    rec.append(r)
    f1.append(f)
    
plt.plot(prs, label = "Test Precision")
plt.plot(rec, label = "Test Recall")
plt.plot(f1, label = "Test F1")

plt.legend()
plt.show()

In [ ]:
prs

In [ ]:
rec